In [1]:
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
# import torch.utils.data
import torch.utils.data as data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data.sampler import SubsetRandomSampler

import cv2
import matplotlib.pyplot as plt
import sys


from copy import deepcopy
from gradCam.VAE_grad_cam import *
from gradCam.vis_util import *

sys.path.append('../classification')
from models.littleConv import littleConv

sys.path.append('../Reconstruction')
from  utils.get_cdim import update_code_dim
from data.dataset import StanfordDog

In [2]:
channel_num = 3
batchsize = 32
traindataset = StanfordDog(root='../Reconstruction/data/', train=True, already = False)
train_loader = torch.utils.data.DataLoader(dataset= traindataset,batch_size=batchsize, shuffle=True)
# val_loader = torch.utils.data.DataLoader(dataset=StanfordDog(root='../Reconstruction/data/', train=True,  already = False), batch_size=batchsize, shuffle=True)
model = littleConv(c_dim=update_code_dim(128, 32, 4), z_dim=200 ,num_channels = channel_num)
model = model.cuda()
picReslu = 32
resume = "../classification/little_.checkpoint.pth.tar"
print("=> loading checkpoint '{}'".format(resume))
checkpoint = torch.load(resume)
start_epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['state_dict'])
print("=> loaded checkpoint '{}' (epoch {})"
  .format(resume, checkpoint['epoch']))

=> loading checkpoint '../classification/little_.checkpoint.pth.tar'
=> loaded checkpoint '../classification/little_.checkpoint.pth.tar' (epoch 13)


In [3]:
for name, module in model.named_modules():
    print (name)


main
main.0
main.1
main.2
main.3
main.4
main.5
main.6
main.7
main.8
main.9
main.10
main.11
linear


In [4]:
from embedding.embedor import convEmbeder

In [5]:
embeder = convEmbeder(model,["main.11"])




NameError: name 'candidate_layers' is not defined

In [ ]:
inpt, target = next(iter(train_loader))
target = target.cuda(non_blocking=True)
inpt = inpt.cuda()
embeder

In [ ]:
np.concatenate